# Initial setup

In [4]:
!nvidia-smi

Fri Sep 23 22:47:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:17:00.0 Off |                  N/A |
| 66%   66C    P0    68W / 260W |      0MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:18:00.0 Off |                  N/A |
| 70%   

In [7]:
import sys
if 'face_utils' in sys.modules:  
    del sys.modules["face_utils"]
    print('face_utils module has been deleted')
from face_utils import *
print('face_utils module has been loaded')

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
face_utils module has been loaded


# Path to datasets:

In [3]:
pathOr     = r'./Dataset/Origin/'
pathDe     = r'./Dataset/Destination/'

# Computation of Saliency Maps

In [4]:
def get_saliencyFast(pathSave,foto,img_A):
    
    img_B = img_A
    
    nh        = 350                # ancho de la mascara Gaussiana
    d         = 40                 # resolución (elimina mascaras de la cara en una grilla de d x d pixeles)
    A         = read_img(img_A)
    Bo        = read_img(img_B)
    
    H0m,H1m   = saliency_minusFast(A,Bo,nh=nh,d=d,n=1,nmod=2,th=0.05)

    D1,Y1     = show_heatmap(Bo,H0m,nh)
    D2,Y2     = show_heatmap(Bo,H1m,nh)

    X         = (D1+D2)/2    # <= HeatMap entre 0 y 1

    Dt,Yt     = show_heatmap(Bo,X,199)
    Dt        = X*255
    
#     Save
    cv2.imwrite(pathSave+"/"+foto[:-4]+'.jpg',Dt)
#     cv2.imwrite(pathSave+"/"+foto[:-4]+'_YtF'+'.jpg',Yt) 

In [4]:
import shutil

def HeatMapMultiFolder(pathA,pathB):
    clases = sorted(os.listdir(pathA))
    for clase in clases[:]:
        print(clase)
        pathC = pathB+clase
        print(pathC)
        if not os.path.exists(pathC): os.mkdir(pathC) # Creamos la carpeta para cada Clase (individuo).
        fotos = sorted(os.listdir(pathA+clase))
        for fotoName in fotos[:]:
            fotoDir = pathA+clase+"/"+fotoName
            if fotoName == ".ipynb_checkpoints":
                shutil.rmtree(fotoDir)
            else:
                get_saliencyFast(pathC,fotoName,fotoDir)
    
HeatMapMultiFolder(pathOr,pathDe)

# Revisar cantidad de carpetas listas. Puede sobrar una que es el checkpoint de Python.

In [3]:
import os

APP_FOLDER = pathDe

totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1

print('Total Number of directories',totalDir)

# Revisar qué carpeta falta.

In [5]:
def HeatMapMultiFolder(pathA,pathB):
    clases = sorted(os.listdir(pathA))
    for clase in clases[:]:
#         print(pathA+clase)
        pathC = pathB+clase
        if len(os.listdir(pathC)) != len(os.listdir(pathA+clase)):
            print(pathC)

HeatMapMultiFolder(pathOr,pathDe)

# Para contar carpetas vacías

In [6]:
n = 0
clases = os.listdir(APP_FOLDER)
for clase in clases:
    dir = APP_FOLDER+clase+"/"
    print(dir)
    break
    if len(os.listdir(dir)) == 0:
        print("Directory is empty")
        print(dir)
        n+=1
print(n)